In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

IMAGE_PATH = 'capture_1756304612.png'

data = np.load(IMAGE_PATH[:IMAGE_PATH.rfind('.')] + '_masks.npz', allow_pickle=True)
masks = data['masks']

image = cv2.imread(IMAGE_PATH)

for mask in masks:
    if mask['area'] < 10000 or mask['area'] > 50000:
        continue

    binary = mask['segmentation'].astype(np.uint8)
    binary = cv2.resize(binary, (image.shape[1], image.shape[0]))

    # plt.figure(figsize=(40,10))
    # plt.subplot(121)
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.subplot(122)
    # plt.imshow(binary, 'gray')
    # plt.show()

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.dilate(binary, kernel, iterations=1)

    cnts, _ = cv2.findContours(
        binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) > 0:     
        cnt = max(cnts, key=cv2.contourArea)

        # cnt = cv2.convexHull(cnt)
        
        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # epsilon = 0.01 * cv2.arcLength(approx, True)
        # approx = cv2.approxPolyDP(approx, epsilon, True)

    img_pts = approx.reshape(-1, 2).astype(np.float32)
    contour_to_draw = approx  # shape: (N,1,2)
    contour = np.zeros(binary.shape, dtype=np.uint8)
    cv2.drawContours(contour, [contour_to_draw], -1, 255, -1)

    processed = image.copy()
    
    # Object overlay
    overlay = np.zeros_like(image, dtype=np.uint8)
    overlay[binary > 0] = (0, 255, 0)  # BGR = червено
    alpha = 0.7  # колко прозрачен да е overlay
    processed = cv2.addWeighted(processed, 1, overlay, alpha, 0)

    
    # комбинираме с оригинала (alpha blending)
    overlay = np.zeros_like(image, dtype=np.uint8)
    overlay[contour > 0] = (0, 0, 255)  # BGR = червено
    alpha = 0.7 # колко прозрачен да е overlay
    processed = cv2.addWeighted(processed, 1, overlay, alpha, 0)
    
    plt.figure(figsize=(40,10))
    plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGR2RGB))
    plt.show()
